# Toronto Neighborhood Analysis
## Clustering Coursera Capstone Project - Week 3 

### Eduardo Palomero-López

### PART 1: Converting Wikipedia Table Data into DataFrame
### PART 2: Add Latitude and Longitude Data
### PART 3: Explore and cluster the Neighborhoods in Toronto

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 24.0MB/s eta 0:00:01     |██████████████████████████████▌ | 5.3MB 24.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

In [4]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

In [5]:
r=requests.get(URL)

### Use BeautifulSoup to parse the data in Wikipedia

In [6]:
parsed_web=bsoup(r.text,"html.parser")
# uncomment next line to check parsing results
#parsed_web

In [7]:
# get table from parsed data
Table=parsed_web.table
# uncomment next line to check results
#Table

In [8]:
results=Table.find_all('tr')
number_rows=len(results)
print('Total rows=',number_rows,'; so total number of data rows (removing header row)=', number_rows-1)

Total rows= 181 ; so total number of data rows (removing header row)= 180


In [9]:
header=results[0].text.split()
header

['Postal', 'Code', 'Borough', 'Neighborhood']

__Check that 'Postal' and 'Code' have been split and need to merge__

In [10]:
header=[header[0]+header[1],header[2],header[3]]
header

['PostalCode', 'Borough', 'Neighborhood']

__Check data__

In [11]:
results[7].text

"\nM7A\n\nDowntown Toronto\n\nQueen's Park, Ontario Provincial Government\n"

In [12]:
results[7].text.split('\n')

['',
 'M7A',
 '',
 'Downtown Toronto',
 '',
 "Queen's Park, Ontario Provincial Government",
 '']

In [13]:
PostalCode=results[7].text.split('\n')[1]
PostalCode

'M7A'

In [14]:
Borough=results[7].text.split('\n')[3]
Borough

'Downtown Toronto'

In [15]:
Neighborhood=results[7].text.split('\n')[5]
Neighborhood

"Queen's Park, Ontario Provincial Government"

In [16]:
# Loop to extract data

Data =[]
n=1
while n < number_rows :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[3]
    Neighborhood=results[n].text.split('\n')[5]
    Data.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(Data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
df.shape

(180, 3)

In [18]:
# Remove rows Borough='Not assigned'
df1=df[~df.Borough.str.contains("Not assigned")]
df1=df1.reset_index(drop=True)
print(df1.shape)
print(df1.head())

(103, 3)
  PostalCode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


In [19]:
distinct_PostalCode = df1['PostalCode'].nunique()
distinct_borough = df1['Borough'].nunique()
distinct_neighbourhood= df1['Neighbourhood'].nunique()
print('Different Postal Codes : ' + str(distinct_PostalCode))
print('Different Boroughs  : '+ str(distinct_borough))
print('Different Neighbourhoods  :' + str(distinct_neighbourhood))

Different Postal Codes : 103
Different Boroughs  : 10
Different Neighbourhoods  :99


In [20]:
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


__Check if any Neighborhood='Not assigned'__

In [21]:
((df1['Neighbourhood'] == 'Not assigned').groupby)

<bound method Series.groupby of 0      False
1      False
2      False
3      False
4      False
       ...  
98     False
99     False
100    False
101    False
102    False
Name: Neighbourhood, Length: 103, dtype: bool>

__There is no data with Neighborhood='Not assigned'__

In [22]:
df1.shape

(103, 3)

## --------------------END OF PART 1----------------------------------

### PART 2: Add Latitude and Longitude Data

In [23]:
# Get coordinates from csv file provided
df_codes=pd.read_csv('http://cocl.us/Geospatial_data')
df_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_codes.shape

(103, 3)

In [25]:
df_codes.columns = ['PostalCode', 'Latitude', 'Longitude']
df_codes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df1sorted=df1.sort_values('PostalCode')
df1sorted.head()

,PostalCode,Borough,Neighbourhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [27]:
NBRHs=pd.merge(df1sorted,df_codes, how='right', on = 'PostalCode')
NBRHs.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [28]:
NBRHs.shape

(103, 5)

## --------------------END OF PART 2----------------------------------

### PART 3: Explore and Cluster the Neighborhoods in Toronto

__Import needed libraries__

In [31]:
import json

#comment or uncomment next row as needed
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#comment or uncomment next row as needed
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map render


__Filter data with Boroughs containing 'Toronto'__

In [32]:
Toronto_filter= NBRHs[NBRHs['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
Toronto_filter.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
Toronto_filter.shape

(39, 5)

__39 Boroughs containing 'Toronto'__

__Get Toronto Coordinates from Google__

In [34]:
lat = 43.6532
long = -79.3832

__Create map of Toronto with Boroughs containing 'Toronto'__

In [35]:

Toronto = folium.Map(location=[lat, long], zoom_start=12)

# Markers
for latit, longit, label in zip(Toronto_filter['Latitude'], Toronto_filter['Longitude'], Toronto_filter['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latit, longit],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
        parse_html=False).add_to(Toronto)  
    
Toronto

__Prepare Foursquare credentials__

In [37]:
CLIENT_ID = 'YPQO0R5GDUG1VMEFVS2DECJC12PN0MG5S4LIGHFKWW0ZX1KY' # Foursquare ID
CLIENT_SECRET = 'YYWVUBTIE0UXQW5AF25EWAL0L3XX2QZDTRJ1CTRG4TPD5OT1' # Foursquare Secret
VERSION = '20180605' # Foursquare API version


__Function to repeat same analysis done for Manhattan__

In [38]:
limit = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

__Code for running the function to get venues and put them in a dataframe__

In [40]:
Toronto_venues = getNearbyVenues(names=Toronto_filter['Neighbourhood'],latitudes=Toronto_filter['Latitude'],longitudes=Toronto_filter['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [41]:
print(Toronto_venues.shape)
Toronto_venues.head(10)

(1613, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
5,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop


__Summary of venues for each Neighborhood__

In [42]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


__Check how many Venue Categories__

In [43]:
print('There are ', len(Toronto_venues['Venue Category'].unique()), ' unique Venue Categories')

There are  239  unique Venue Categories


__For Neighbour Analysis, we start with a table that pivots individual venues per neighbourhood__

In [44]:
# Create table with Venues
Toronto_venuexNBRH = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column
Toronto_venuexNBRH['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(Toronto_venuexNBRH.columns.values)
cols.pop(cols.index('Neighborhood'))
Toronto_venuexNBRH=Toronto_venuexNBRH[['Neighborhood']+cols]

print(Toronto_venuexNBRH.shape)
Toronto_venuexNBRH.head(10)

(1613, 239)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category__

In [45]:
Toronto_groupBy = Toronto_venuexNBRH.groupby('Neighborhood').mean().reset_index()
Toronto_groupBy

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.015873,0.000000,0.015873,0.000000,0.015873,0.000000,0.000000,0.015873
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
Toronto_groupBy.shape

(39, 239)

__Top 5 most common venues per neighborhood__

In [47]:
#Toronto_venuexNBRH

top_venues = 5

for hood in Toronto_groupBy['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_groupBy[Toronto_groupBy['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2         Cheese Shop  0.04
3  Seafood Restaurant  0.04
4                 Pub  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3   Burrito Place  0.05
4    Intersection  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1         Pizza Place  0.06
2             Butcher  0.06
3          Restaurant  0.06
4       Auto Workshop  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.13
1            Plane  0.07
2          Airport  0.07
3      Coffee Shop  0.07
4              Bar  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.1

__Function to sort the venues in descending order__

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

__New dataframe and display the top 10 venues for each neighborhood__

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_groupBy['Neighborhood']
for ind in np.arange(Toronto_groupBy.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_groupBy.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Café,Bakery,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Harbor / Marina,Bar,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique,Airport Lounge
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Japanese Restaurant,Department Store,Salad Place,Bubble Tea Shop,Ice Cream Shop


## Cluster Neighbourhoods

__Use k-means to cluster the neighborhood into 5 clusters__

In [50]:
# set number of clusters
kclusters = 5
Toronto_groupBy_cluster = Toronto_groupBy.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_groupBy_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

__New dataframe that includes the cluster as well as the top 10 venues for each neighborhood__

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_filter
#Toronto_merged.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)

# merge Toronto_groupBy with Toronto_filter to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Coffee Shop,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Liquor Store,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Sandwich Place,Fast Food Restaurant,Pet Store,Pub,Burrito Place,Board Shop,Italian Restaurant,Fish & Chips Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [61]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], 
                                  Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

    
map_clusters

## Examining Clusters

__Note that K-means clustering resulting in most neighborhoods in cluster 0, while clusters 1, 2, 3 and 4 containing only 1 neighborhood each__

__Cluster 0__

In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Health Food Store,Coffee Shop,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Liquor Store,Spa,Juice Bar
2,East Toronto,0,Park,Sandwich Place,Fast Food Restaurant,Pet Store,Pub,Burrito Place,Board Shop,Italian Restaurant,Fish & Chips Shop,Steakhouse
3,East Toronto,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
5,Central Toronto,0,Gym,Hotel,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gym / Fitness Center,General Travel
6,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Park
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Pharmacy,Seafood Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Bank,Restaurant,Liquor Store,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Pizza Place,American Restaurant
11,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Café,Bakery,Restaurant,Chinese Restaurant,Sandwich Place,Playground
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel


__Cluster 1__

In [63]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Gym,Trail,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


__Cluster 2__

In [64]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


__Cluster 3__

In [65]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Garden,Home Service,Music Venue,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


__Cluster 4__

In [66]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
